In [24]:
# from functions import *
import random
import secrets
import pandas as pd
import numpy as np
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend

In [25]:
## Constants

constants_eip1559 = {
    "BASEFEE_MAX_CHANGE_DENOMINATOR": 8,
    "TARGET_SIZE": 12500000,
    "MAX_BLOCK_SIZE": 25000000
}

constants_legacy = {
    "MAX_BLOCK_SIZE": 12500000,
}

levers = {
    "mempool_length": 100,
    "INITIAL_BASEFEE": 1 * (10 ** 9),
}

In [26]:
## Classes

class eip1559_transaction:
    def __init__(self,total_gas,fee_cap,premium):
        
        # Inherent txn propoerties 
        self.total_gas_used = total_gas
        self.transaction_hash = secrets.token_bytes(8)
        
        # User adjusted txn properties
        self.fee_cap = fee_cap
        self.premium = premium
        
class legacy_transaction:
    def __init__(self,total_gas,fee):
        
        # Inherent txn properties
        self.total_gas_used = total_gas
        self.transaction_hash = secrets.token_bytes(8)
        
        # User adjusted txn properties 
        self.fee = fee
        
class block:
    def __init__(self,size,transactions):
        self.size = size
        self.transactions = transactions 

In [27]:
## Functions 

def demand_functions_eip1559(params, substep, state_history, previous_state, policy_input):
    
    number_of_transactions_in_mempool = levers['mempool_length']
    demand_dict = {}
    
    a = random.randint(1,3)
    b = random.randint(3,7)
    
    for i in range(number_of_transactions_in_mempool):
        tx = eip1559_transaction(
            total_gas = random.randint(20000,30000),
            premium = a*(10**9),
            fee_cap = b *(10**9)
        )
        demand_dict[tx.transaction_hash] = tx
        
        print("New transaction {}, submitted into the mempool".format(tx.transaction_hash))
    
    return ("demand", demand_dict)

def demand_functions_legacy(params, substep, state_history, previous_state, policy_input):
    
    number_of_transactions_in_mempool = levers['mempool_length']
    demand_dict = {}
    
    #a = random.randint(1,3)
    b = random.randint(3,7)
    
    for i in range(number_of_transactions_in_mempool):
        tx = legacy_transaction(
            total_gas = random.randint(20000,30000),
            #premium = a*(10**9),
            fee = b * (10 ** 9)
        )
        demand_dict[tx.transaction_hash] = tx
        
        print("New transaction {}, submitted into the mempool".format(tx.transaction_hash))
    
    return ("demand", demand_dict)


def transaction_selection_eip1559(params, substep, state_history, previous_state):
    
    demand_dict = previous_state["demand"]
    basefee = previous_state["basefee"]
    
    size = constants_eip1559["MAX_BLOCK_SIZE"]
    final_fee_transactions = {}
    
    for i in demand_dict.keys():
        if basefee > demand_dict[i].fee_cap:
            continue
        else:
            final_fee_transactions[i] = min(demand_dict[i].premium - basefee, demand_dict[i].fee_cap)
            
    final_fee_transactions = dict(sorted(final_fee_transactions.items(), key=lambda x: x[1], reverse=True))
    
    included_transactions = []
    total_size_used = 0
    
    for i in final_fee_transactions.keys():
        
        if total_size_used < size:
            included_transactions += [demand_dict[i]]
            
        total_size_used += demand_dict[i].total_gas_used
        
    print("{} Number of transactions being included in the next block".format(len(included_transactions)))
        
    return { "block": block(size = total_size_used, transactions = included_transactions) }
        
def transaction_selection_legacy(params, substep, state_history, previous_state):
    
    demand_dict = previous_state["demand"]
    
    size = constants_legacy["MAX_BLOCK_SIZE"]
    final_fee_transactions = {}
    
    for i in demand_dict.keys():
            final_fee_transactions[i] =  demand_dict[i].fee     
            
    final_fee_transactions = dict(sorted(final_fee_transactions.items(), key=lambda x: x[1], reverse=True))
    
    included_transactions = []
    total_size_used = 0
    
    for i in final_fee_transactions.keys():
        
        if total_size_used < size:
            included_transactions += [demand_dict[i]]
            
        total_size_used += demand_dict[i].total_gas_used
          
    print("{} Number of transactions being included in the next block".format(len(included_transactions)))
        
    return { "block": block(size = total_size_used, transactions = included_transactions) }
        
def update_basefee(params, substep, state_history, previous_state, policy_input):
    
    gas_used = sum([i.total_gas_used for i in policy_input["block"].transactions])
    basefee = previous_state["basefee"]
    
    basefee = basefee + basefee * (gas_used - constants_eip1559["TARGET_SIZE"]) // constants_eip1559["TARGET_SIZE"] // constants_eip1559["BASEFEE_MAX_CHANGE_DENOMINATOR"]
    
    print("The new basefee is: {}".format(basefee))
          
    return ("basefee", basefee)


def record_latest_block(params, substep, state_history, previous_state, policy_input):
    
    block = policy_input["block"]
          
    print("New block recorded")
    
    return ("latest_block", block)

## BASELINE MODEL

### Legacy

In [28]:
psub = [{
    "policies": {},
    "variables": {
        "demand": demand_functions_legacy # step 1
    }
}, {
    "policies": {
        "action": transaction_selection_legacy # step 2
    },
    "variables": {
        "latest_block": record_latest_block # step 3
    }
}]

initial_state = {
    "demand": {},
    "latest_block": block(size = 0, transactions=[])
}

blocks = 30

model = Model(
    initial_state=initial_state,
    state_update_blocks=psub,
)
simulation = Simulation(model=model, timesteps=blocks, runs=1)
experiment = Experiment([simulation])
experiment.engine = Engine(deepcopy=False, drop_substeps=True)
result = experiment.run()
df1 = pd.DataFrame(result)

### EIP1559

In [29]:
psub = [{
    "policies": {},
    "variables": {
        "demand": demand_functions_eip1559 # step 1
    }
}, {
    "policies": {
        "action": transaction_selection_eip1559 # step 2
    },
    "variables": {
        "basefee": update_basefee, # step 3
        "latest_block": record_latest_block 
    }
}]

initial_state = {
    "demand": {},
    "latest_block": block(size = 0, transactions=[]),
    "basefee": 10 * (10 ** 9)
}

blocks = 30

model = Model(
    initial_state=initial_state,
    state_update_blocks=psub,
)
simulation = Simulation(model=model, timesteps=blocks, runs=1)
experiment = Experiment([simulation])
experiment.engine = Engine(deepcopy=False, drop_substeps=True)
result = experiment.run()
df2 = pd.DataFrame(result)

### Plots

In [30]:
df1

,demand,latest_block,simulation,subset,run,substep,timestep
0,{},<__main__.block object at 0x000002579785AFA0>,0,0,1,0,0
1,{b'x\xca\xda\xde2\xfe\xf3\xf9': <__main__.lega...,<__main__.block object at 0x0000025797800A00>,0,0,1,2,1
2,{b'T%\x14O\x96\xfed\xf2': <__main__.legacy_tra...,<__main__.block object at 0x0000025797764250>,0,0,1,2,2
3,{b'\xe7\xfbJ\x97\xe3\xaa}\x83': <__main__.lega...,<__main__.block object at 0x0000025797771C40>,0,0,1,2,3
4,{b'\xffN\x15[\x0b\x0f\x08\xd6': <__main__.lega...,<__main__.block object at 0x00000257977D2550>,0,0,1,2,4
5,"{b""\x9b('/W^\\\xd8"": <__main__.legacy_transact...",<__main__.block object at 0x00000257977B7F10>,0,0,1,2,5
6,{b'\x0c\xf3eQ\xb8\x80\xa6\xd2': <__main__.lega...,<__main__.block object at 0x0000025797864850>,0,0,1,2,6
7,{b'\xd5\xe8}\r\x0cm\xd7\xe8': <__main__.legacy...,<__main__.block object at 0x00000257978721F0>,0,0,1,2,7
8,{b'<\n\x8e\xc0\xe2\x9a\xa5\xf4': <__main__.leg...,<__main__.block object at 0x000002579787DB50>,0,0,1,2,8
9,{b'\x17*\xa1\x0f]\xda\x0b\xaa': <__main__.lega...,<__main__.block object at 0x000002579788B4F0>,0,0,1,2,9


In [31]:
df2

,demand,latest_block,basefee,simulation,subset,run,substep,timestep
0,{},<__main__.block object at 0x00000257978084C0>,10000000000,0,0,1,0,0
1,{b'\xaeku\x15\xb3\xb2L\r': <__main__.eip1559_t...,<__main__.block object at 0x00000257977AAE20>,8750000000,0,0,1,2,1
2,{b'wkk\xd6V4\x96E': <__main__.eip1559_transact...,<__main__.block object at 0x00000257977757C0>,7656250000,0,0,1,2,2
3,"{b'\x1f-H\xba""#/\xc7': <__main__.eip1559_trans...",<__main__.block object at 0x000002579796E160>,6699218750,0,0,1,2,3
4,{b'\x0f\xb8\xb6\x04\x11\xc4\xd3\x15': <__main_...,<__main__.block object at 0x0000025797989AC0>,6029023010,0,0,1,2,4
5,{b'\xba\xd9\x11\xfb=\xaaA\xbd': <__main__.eip1...,<__main__.block object at 0x00000257978F3460>,5426765513,0,0,1,2,5
6,{b't\x02\xbd_\xd7\xfc\x8e\x92': <__main__.eip1...,<__main__.block object at 0x000002579790ADC0>,4883425702,0,0,1,2,6
7,{b'\x19cL\x0b\xc4\x7faQ': <__main__.eip1559_tr...,<__main__.block object at 0x0000025797870760>,4272997489,0,0,1,2,7
8,{b'L\xc2\xf8\xde\xce\xec!\x1e': <__main__.eip1...,<__main__.block object at 0x0000025797890100>,3847334853,0,0,1,2,8
9,"{b""\xd89p\x14e\x9c'!"": <__main__.eip1559_trans...",<__main__.block object at 0x00000257978AAA60>,3464058276,0,0,1,2,9
